In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [2]:
!mkdir -p drive
!google-drive-ocamlfuse drive
!pip install -q keras
!pip install --upgrade google-cloud-bigquery

    100% |████████████████████████████████| 81kB 4.6MB/s 
  Found existing installation: google-cloud-bigquery 1.1.0
    Uninstalling google-cloud-bigquery-1.1.0:
      Successfully uninstalled google-cloud-bigquery-1.1.0


In [9]:
#BigQuery
from google.cloud import bigquery
import os
import pandas 
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "drive/CloudAI/reditchatbot-7d62160d61c5.json"

bigquery_client = bigquery.Client()

#,a.parent_id,b.id
query = """SELECT body, title,a.score, a.subreddit
FROM `fh-bigquery.reddit_comments.2018_05` as a
JOIN `fh-bigquery.reddit_posts.2018_05`  as b
ON a.parent_id=CONCAT('t3_',b.id) 
where a.score = 
(select max(a.score) 
    from `fh-bigquery.reddit_comments.2018_05` as c
   where c.parent_id = a.parent_id and c.link_id=c.parent_id) 
and a.score >20 and LENGTH(a.body)>0 and LENGTH(a.body)<70 and 
LENGTH(b.title)>0 and LENGTH(b.title)<70 and 
a.body != '[removed]' and a.body != '[deleted]' 
and b.title != '[removed]' and b.title != '[deleted]' and a.body != b.title
LIMIT 5000000
"""
query_job = bigquery_client.query(query)

df = query_job.to_dataframe()
# have format
# comment_text, reply_text,score, subject
df.rename(columns={'body': 'reply_text', 'title': 'comment_text'}, inplace=True)
df['reply_text'] = df.reply_text.str.replace('\n' , '')
df['reply_text'] = df.reply_text.str.replace('"' , '')
df['comment_text'] = df.comment_text.str.replace('\n' , '')
df['comment_text'] = df.comment_text.str.replace('"' , '')
print(df.head(10))
df.to_csv('drive/CloudAI/reddit_data_set.csv', sep='\t')

                                          reply_text  \
0  Great, Nuns with automatic weapons. Guess that...   
1   source: https://www.artstation.com/artwork/KmdZ9   
2  This is the quality that has been missing for ...   
3                        &gt; Got'em -Johnny Russell   
4                            DRAGON KICK OP PLS NERF   
5  This is the best shitty superpower I've ever seen   
6  I don't think I've ever seen cat fur come so c...   
7                     What's in the box?!- Brad Pitt   
8                  Mr. Admiral. I don't feel so good   
9        What a terrible end of the season for Demar   

                                        comment_text  score subreddit  
0                  photo Study, Digital, 2480x2512px    256       Art  
1                 Betty, Qi Sheng Luo, Digital, 2018    256       Art  
2                       A Nightmare on Legacy Street    256       Eve  
3  Sporting KC's team photo's have been really fu...    256       MLS  
4           [Spoiler] V

In [16]:
df = pandas.read_csv('drive/CloudAI/reddit_data_set.csv', sep='\t')
print(df.head(200))

    Unnamed: 0                                         reply_text  \
0            0  Great, Nuns with automatic weapons. Guess that...   
1            1   source: https://www.artstation.com/artwork/KmdZ9   
2            2  This is the quality that has been missing for ...   
3            3                        &gt; Got'em -Johnny Russell   
4            4                            DRAGON KICK OP PLS NERF   
5            5  This is the best shitty superpower I've ever seen   
6            6  I don't think I've ever seen cat fur come so c...   
7            7                     What's in the box?!- Brad Pitt   
8            8                  Mr. Admiral. I don't feel so good   
9            9        What a terrible end of the season for Demar   
10          10                             #HERE COMES CHRIS PAUL   
11          11           Lmao his foot went higher than CP3s head   
12          12  Goodness they are unbeatable when they play li...   
13          13  The /r/Kings subre

In [4]:
# testing and training
# change like it is for nmt-chat bot tst2012 and tst2013 both only using 100 comments, strange do
from sklearn.cross_validation import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(df['comment_text'].values, df['reply_text'].values, test_size=0.25)
with open("drive/CloudAI/train.from",'a',encoding="utf-8") as f:
	for content in xtrain:
		f.write(str(content)+'\n')

with open("drive/CloudAI/test.from",'a',encoding="utf-8") as f:
	for content in xtest:
		f.write(str(content)+'\n')

with open("drive/CloudAI/test.to",'a',encoding="utf-8") as f:
	for content in ytest:
		f.write(str(content)+'\n')

with open("drive/CloudAI/train.to",'a',encoding="utf-8") as f:
	for content in ytrain:
		f.write(str(content)+'\n')

/usr/local/lib/python3.6/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
!pip install -r drive/CloudAI/nmt-chatbot/requirements.txt 
!pip install --upgrade tensorflow-gpu==1.4

    100% |████████████████████████████████| 253.3MB 97kB/s 
tcmalloc: large alloc 1073750016 bytes == 0x4f676000 @  0x7f83570f21c4 0x46d6a4 0x5fcbcc 0x4c494d 0x54f3c4 0x553aaf 0x54e4c8 0x54f4f6 0x553aaf 0x54efc1 0x54f24d 0x553aaf 0x54efc1 0x54f24d 0x553aaf 0x54efc1 0x54f24d 0x551ee0 0x54e4c8 0x54f4f6 0x553aaf 0x54efc1 0x54f24d 0x551ee0 0x54efc1 0x54f24d 0x551ee0 0x54e4c8 0x54f4f6 0x553aaf 0x54e4c8
    100% |████████████████████████████████| 51kB 14.9MB/s 
    100% |████████████████████████████████| 634kB 7.0MB/s 
    100% |████████████████████████████████| 51kB 14.4MB/s 
    100% |████████████████████████████████| 3.3MB 2.5MB/s 
  Running setup.py bdist_wheel for regex ... - \ | / - \ done
  Stored in directory: /content/.cache/pip/wheels/cc/bf/b9/a2c431d3b6285626a4938ee6dbd287b961e5142c5ab80b8e8b
  Running setup.py bdist_wheel for python-Levenshtein ... - \ | done
  Stored in directory: /content/.cache/pip/wheels/de/c2/93/660fd5f7559049268ad2dc6d81c4e39e9e36518766eaf

In [0]:
!python drive/CloudAI/nmt-chatbot/setup/prepare_data.py


Preparing training set from raw set
File: train.from


In [11]:
%cd drive/CloudAI/nmt-chatbot
!python train.py

[Errno 2] No such file or directory: 'drive/CloudAI/nmt-chatbot'
/content/drive/CloudAI/nmt-chatbot
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


Training model...


Epoch: 1, steps per epoch: 1525, epoch ends at 1525 steps, learning rate: 0.001 - training

hi
# Job id 0
# hparams:
  src=from
  tgt=to
  train_prefix=data/train.bpe
  dev_prefix=data/tst2012.bpe
  test_prefix=data/tst2013.bpe
  out_dir=model/
# Vocab file data/vocab.bpe.from exists
  using source vocab for target
  saving hparams to model/hparams
  saving hparams to model/best_bleu/hparams
  attention=scaled_luong
  attention_architecture=standard
  avg_ckpts=False
  batch_size=128
  beam_width=20
  best_bleu=0
  best_bleu_dir=model/best_bleu
  check_special_token=True
  colocate_gradients_with_ops=True
  decay_scheme=
  dev_prefix=data/tst2012.bpe
  dropout=0.2

  created eval model with fresh parameters, time 0.17s
  eval dev: perplexity 16627.84, time 1s, Sun Jul 29 09:38:07 2018.
2018-07-29 09:38:10.218336: W tensorflow/core/framework/op_kernel.cc:1192] Out of range: End of sequence
	 [[Node: IteratorGetNext = IteratorGetNext[output_shapes=[[?,?], [?,?], [?,?], [?], [?]], output_types=[DT_INT32, DT_INT32, DT_INT32, DT_INT32, DT_INT32], _device="/job:localhost/replica:0/task:0/device:CPU:0"](Iterator)]]
  eval test: perplexity 15854.35, time 2s, Sun Jul 29 09:38:10 2018.
2018-07-29 09:38:10.310365: W tensorflow/core/kernels/lookup_util.cc:362] Table trying to initialize from file data/vocab.bpe.from is already initialized.
2018-07-29 09:38:10.314204: W tensorflow/core/kernels/lookup_util.cc:362] Table trying to initialize from file data/vocab.bpe.from is already initialized.
  created infer model with fresh parameters, time 0.09s
# Start step 0, lr 0.001, Sun Jul 29 09:38:10 2018
# Init train iterator, skipping 0 elements
2018-07-29 09:38:20

# Finished an epoch, step 1180. Perform external evaluation
2018-07-29 10:51:41.181419: W tensorflow/core/kernels/lookup_util.cc:362] Table trying to initialize from file data/vocab.bpe.from is already initialized.
2018-07-29 10:51:41.184868: W tensorflow/core/kernels/lookup_util.cc:362] Table trying to initialize from file data/vocab.bpe.from is already initialized.
  loaded infer model parameters from model/translate.ckpt-1000, time 1.47s
  # 79
    src: Imgur. It hasn't been the same for the last 2 years
    ref: Imgur. It hasn't been the same for the last 2 years
    nmt: http://www.youtube.com/watch?v=s_s_s_s_t_status/8444
2018-07-29 10:51:42.941359: W tensorflow/core/kernels/lookup_util.cc:362] Table trying to initialize from file data/vocab.bpe.from is already initialized.
2018-07-29 10:51:42.947434: W tensorflow/core/kernels/lookup_util.cc:362] Table trying to initialize from file data/vocab.bpe.from is already initialized.
  loaded infer model parameters from model/translate.c

2018-07-29 11:16:06.014904: W tensorflow/core/kernels/lookup_util.cc:362] Table trying to initialize from file data/vocab.bpe.from is already initialized.
  created infer model with fresh parameters, time 0.10s
  # 86
    src: Led Zeppelin intentionally misspelled Lead as Led. They were concerned that Lead would be pronounced like The Warriors blew a 3-1 Lead to the Cavs rather than Lead like There's too much lead in the water supply
    ref: Led Zeppelin intentionally misspelled Lead as Led. They were concerned that Lead would be pronounced like The Warriors blew a 3-1 Lead to the Cavs rather than Lead like There's too much lead in the water supply
    nmt: justjust grain grain trailers trailers trailers trailers trailers trailers attempt attempt attempt attempt attempt attempt attempt attempt attemptwritingwritingwritingwritingwritingawwawwawwawwawwunityunityunityunityunity heroin heroin heroinlicoplicoplicoplicoplicoplicop?? CO CO CO CO CO CO CO COalgalgalgalg ........ ........ ....

# Trainable variables
  embeddings/embedding_share:0, (15003, 512), /device:GPU:0
  dynamic_seq2seq/encoder/bidirectional_rnn/fw/basic_lstm_cell/kernel:0, (1024, 2048), /device:GPU:0
  dynamic_seq2seq/encoder/bidirectional_rnn/fw/basic_lstm_cell/bias:0, (2048,), /device:GPU:0
  dynamic_seq2seq/encoder/bidirectional_rnn/bw/basic_lstm_cell/kernel:0, (1024, 2048), /device:GPU:0
  dynamic_seq2seq/encoder/bidirectional_rnn/bw/basic_lstm_cell/bias:0, (2048,), /device:GPU:0
  dynamic_seq2seq/decoder/memory_layer/kernel:0, (1024, 512), 
  dynamic_seq2seq/decoder/attention/multi_rnn_cell/cell_0/basic_lstm_cell/kernel:0, (1536, 2048), /device:GPU:0
  dynamic_seq2seq/decoder/attention/multi_rnn_cell/cell_0/basic_lstm_cell/bias:0, (2048,), /device:GPU:0
  dynamic_seq2seq/decoder/attention/multi_rnn_cell/cell_1/basic_lstm_cell/kernel:0, (1024, 2048), /device:GPU:0
  dynamic_seq2seq/decoder/attention/multi_rnn_cell/cell_1/basic_lstm_cell/bias:0, (2048,), /device:GPU:0
  dynamic_seq2seq/decoder/atten

...

2018-07-29 12:30:07.641550: W tensorflow/core/kernels/lookup_util.cc:362] Table trying to initialize from file data/vocab.bpe.from is already initialized.
2018-07-29 12:30:07.642681: W tensorflow/core/kernels/lookup_util.cc:362] Table trying to initialize from file data/vocab.bpe.from is already initialized.
  loaded infer model parameters from model/translate.ckpt-2525, time 1.46s
# External evaluation, global step 2525
  decoding to output model/output_dev_2525.
2018-07-29 12:30:18.374058: W tensorflow/core/framework/op_kernel.cc:1192] Out of range: End of sequence
	 [[Node: IteratorGetNext = IteratorGetNext[output_shapes=[[?,?], [?]], output_types=[DT_INT32, DT_INT32], _device="/job:localhost/replica:0/task:0/device:CPU:0"](Iterator)]]
2018-07-29 12:30:18.374214: W tensorflow/core/framework/op_kernel.cc:1192] Out of range: End of sequence
	 [[Node: IteratorGetNext = IteratorGetNext[output_shapes=[[?,?], [?]], output_types=[DT_INT32, DT_INT32], _device="/job:localhost/replica:0/task:

    src: http://i.imgur.com/UlpgzF2.jpg
    ref: http://i.imgur.com/UlpgzF2.jpg
    nmt: ifaNONONO supply supply supply supply supply supply supply male male male male viewsridridrid ren ren shitpost shitpost བ བ བ བ བ བ བ བ བ transition transition transition influ
2018-07-29 12:53:21.011500: W tensorflow/core/kernels/lookup_util.cc:362] Table trying to initialize from file data/vocab.bpe.from is already initialized.
  created eval model with fresh parameters, time 0.15s
  eval dev: perplexity 16373.23, time 0s, Sun Jul 29 12:53:21 2018.
  eval test: perplexity 16266.21, time 0s, Sun Jul 29 12:53:22 2018.
2018-07-29 12:53:23.023530: W tensorflow/core/kernels/lookup_util.cc:362] Table trying to initialize from file data/vocab.bpe.from is already initialized.
2018-07-29 12:53:23.027635: W tensorflow/core/kernels/lookup_util.cc:362] Table trying to initialize from file data/vocab.bpe.from is already initialized.
  created infer model with fresh parameters, time 0.11s
# Best bleu, step 0 l

  # 21
    src: He didn't include the words heart or light ten times. Inaccurate.
    ref: He didn't include the words heart or light ten times. Inaccurate.
    nmt: [deleted]
  loaded eval model parameters from model/translate.ckpt-3050, time 1.52s
  eval dev: perplexity 180.77, time 1s, Sun Jul 29 12:53:41 2018.
  eval test: perplexity 168.35, time 1s, Sun Jul 29 12:53:42 2018.
2018-07-29 12:53:44.272519: W tensorflow/core/kernels/lookup_util.cc:362] Table trying to initialize from file data/vocab.bpe.from is already initialized.
2018-07-29 12:53:44.274366: W tensorflow/core/kernels/lookup_util.cc:362] Table trying to initialize from file data/vocab.bpe.from is already initialized.
  loaded infer model parameters from model/translate.ckpt-3050, time 1.52s
# External evaluation, global step 3050
  decoding to output model/output_dev_3050.
2018-07-29 12:53:52.097686: W tensorflow/core/framework/op_kernel.cc:1192] Out of range: End of sequence
	 [[Node: IteratorGetNext = IteratorGetNext

  step 3750 lr 1e-05 step-time 3.52s wps 1.71K ppl 96.89 gN 10.70 bleu 0.00, Sun Jul 29 13:36:26 2018
  step 3850 lr 1e-05 step-time 3.67s wps 1.65K ppl 98.42 gN 10.89 bleu 0.00, Sun Jul 29 13:42:34 2018
  step 3950 lr 1e-05 step-time 3.50s wps 1.71K ppl 98.30 gN 10.75 bleu 0.00, Sun Jul 29 13:48:24 2018
  step 4050 lr 1e-05 step-time 3.71s wps 1.64K ppl 98.24 gN 10.94 bleu 0.00, Sun Jul 29 13:54:35 2018
# Save eval, global step 4050
2018-07-29 13:55:33.103023: W tensorflow/core/kernels/lookup_util.cc:362] Table trying to initialize from file data/vocab.bpe.from is already initialized.
2018-07-29 13:55:33.105527: W tensorflow/core/kernels/lookup_util.cc:362] Table trying to initialize from file data/vocab.bpe.from is already initialized.
  loaded infer model parameters from model/translate.ckpt-4050, time 1.54s
  # 38
    src: Anyone know why this was removed from /r/politics. This story seems huge and was on top of all just minutes ago. If they're going to censor this, you guys know w

2018-07-29 14:29:39.942607: W tensorflow/core/framework/op_kernel.cc:1192] Out of range: End of sequence
	 [[Node: IteratorGetNext = IteratorGetNext[output_shapes=[[?,?], [?,?], [?,?], [?], [?]], output_types=[DT_INT32, DT_INT32, DT_INT32, DT_INT32, DT_INT32], _device="/job:localhost/replica:0/task:0/device:CPU:0"](Iterator)]]
  eval test: perplexity 169.83, time 1s, Sun Jul 29 14:29:39 2018.
2018-07-29 14:29:41.410908: W tensorflow/core/kernels/lookup_util.cc:362] Table trying to initialize from file data/vocab.bpe.from is already initialized.
2018-07-29 14:29:41.420045: W tensorflow/core/kernels/lookup_util.cc:362] Table trying to initialize from file data/vocab.bpe.from is already initialized.
  loaded infer model parameters from model/translate.ckpt-4575, time 1.47s
# External evaluation, global step 4575
  decoding to output model/output_dev_4575.
2018-07-29 14:29:50.348601: W tensorflow/core/framework/op_kernel.cc:1192] Out of range: End of sequence
	 [[Node: IteratorGetNext = It

In [14]:
# create chat text
%cd drive/CloudAI/nmt-chatbot
file = open("testfile.txt","w") 
 
file.write("Hello World\n") 
file.write("roast me\n") 
file.write("what is my name\n") 
file.write("can you help me\n")
 
file.close() 

[Errno 2] No such file or directory: 'drive/CloudAI/nmt-chatbot'
/content/drive/CloudAI/nmt-chatbot


In [15]:
%cd drive/CloudAI/nmt-chatbot
!python inference.py < testfile.txt

[Errno 2] No such file or directory: 'drive/CloudAI/nmt-chatbot'
/content/drive/CloudAI/nmt-chatbot
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
https://www.reddit.com/r/pics/comments/1010/101
http://i.imgur.com/1n1nR.gifv
This is one of the best things I've ever seen.
http://i.imgur.com/1K111.jpg


In [4]:
# deployment
%cd drive/CloudAI/nmt-chatbot
!python utils/prepare_for_deployment.py

/content/drive/CloudAI/nmt-chatbot


List of available checkpoints:
1. *translate.ckpt-4575
2. translate.ckpt-4050
3. translate.ckpt-3050
4. translate.ckpt-2525
5. translate.ckpt-1525
6. translate.ckpt-1000
quit. Quit tool
select default
Copying: core
Copying: core/scorer.py
Copying: core/tokenizer.py
Copying: core/sentence.py
Copying: core/__init__.py
Copying: nmt
Copying: nmt/README.md
Copying: nmt/CONTRIBUTING.md
Copying: nmt/LICENSE
Copying: nmt/nmt
Copying: nmt/nmt/attention_model.py
Copying: nmt/nmt/train.py
Copying: nmt/nmt/nmt_test.py
Copying: nmt/nmt/gnmt_model.py
Copying: nmt/nmt/nmt.py
Copying: nmt/nmt/model_helper.py
Copying: nmt/nmt/inference.py
Copying: nmt/nmt/model.py
Copying: nmt/nmt/inference_test.py
Copying: nmt/nmt/model_test.py
Copying: nmt/nmt/__init__.py
Copying: nmt/nmt/utils
Copying: nmt/nmt/scripts
Copying: nmt/nmt/standard_hparams
Copying: nmt/nmt/testdata
Copying: nmt/nmt/g3doc
Copying: nmt/nmt/utils/iterator_utils.py
Copying: nmt/nmt/utils/vocab_utils.py
Co